# One Dimensional Hard Spheres

In [1]:
import time
import numpy as np
import matplotlib.pyplot as plt
import plotly.offline as py
from plotly.graph_objs import Layout, Scatter, Heatmap, Histogram, Figure

In [2]:
def sample2(sampling_function, n_samples, L, sigma):
    samples1 = np.empty(n_samples)
    samples2 = np.empty(n_samples)
    
    for i in range(n_samples):
        p1, p2 = sampling_function(L, sigma)
        samples1[i] = p1
        samples2[i] = p2

    return samples1, samples2 


def sample2_naive(L, sigma):
    while True:
        p1 = np.random.uniform(0 + sigma, L - sigma)
        p2 = np.random.uniform(0 + sigma, L - sigma)

        if abs(p1 - p2) > 2*sigma:
            return p1, p2


def sample2_wrong(L, sigma):
    while True:
        p1 = np.random.uniform(0 + sigma, L - sigma)
        
        while True:
            p2 = np.random.uniform(0 + sigma, L - sigma)
            if abs(p1 - p2) > 2*sigma:
                return p1, p2


class PinSampler(object):
    def __init__(self, num_spheres, length, sigma):
        self.num_spheres = num_spheres
        self.length = length
        self.sigma = sigma

        if num_spheres * 2*sigma >= length:
            raise Exception("Invalid number of spheres!")

    def sample(self, size=1):
        s = np.empty((size, self.num_spheres))
        
        for i in range(size):
            s[i] = self._sample()

        return s


class DirectPinSampler(PinSampler):
    def _sample(self):
        s = np.empty(self.num_spheres)

        i = 0
        while True:
            p = np.random.uniform(self.sigma, self.length - self.sigma)

            if self._overlap(p, s):
                s = np.empty(self.num_spheres)
                i = 0

            else:
                s[i] = p
                i += 1

                if i >= self.num_spheres:
                    return s


    def _overlap(self, p, sample):
        return np.any(np.abs(sample - p) < 2*self.sigma)
    
class InflateDeflatePinSampler(PinSampler):
    def _sample(self):
        # Deflated sample
        deflated = sorted(np.random.uniform(
            0,
            self.length - 2*self.num_spheres*self.sigma,
            size=self.num_spheres
        ))

        # Inflate
        sample = np.empty(self.num_spheres)
        for i, x in enumerate(deflated):
            sample[i] = x + (2*i + 1)*self.sigma

        return sample

## 1 Naive direct sampling of two balls

The purpose of the exercise is sampling uniformly the space of configurations of two hard spheres in 1 dimension. The parameters of the problem are:
$2\sigma$: width of one sphere
$L$: space between the walls that bounds the allowed space for the sheres
$n$: number of spheres.

We first look at the problem with two spheres with the following parameters: $\sigma=0.75$, $L=8$.  
The 'Naive direct sampling' is a simple accept/reject method in which, if the spheres overlap, both of them are thrown away and sampled again. As we can see from the histogram the positions near the walls are more probable than the middle ones. As shown during the lectures the ratio between the maximum and minimum probabilities should be:

$$
\begin{equation}
r=\dfrac{L-4\sigma}{L-6\sigma}=\dfrac{7-4\cdot 0.75}{8-6\cdot 0.75}= 0.7
\end{equation}
$$

and this prediction is on average fullfilled by the data portrayed in the histogram.

In [3]:
y=sample2(sample2_naive, 10**5, 8, 0.75)
data = [
    Histogram(x=y[0], histnorm="probability", name="sphere1"),
    Histogram(x=y[1], histnorm="probability", name="sphere2")
]

lyt = Layout(title="Naive direct sampling", xaxis=dict(title="pdf sphere"),
             bargroupgap=0.1)

py.plot(Figure(data=data, layout=lyt))

'file://C:\\Users\\Diego\\Desktop\\computational-science\\h4\\temp-plot.html'

## 2 Wrong direct sampling of two balls

We try to sample the configurations throwing away just the second sphere in the case of an overlap. 
We see that the configuration space of the first particle is not sampled correctly being a trivial uniform distribution in $[\sigma, L-\sigma]$.

In [ ]:
z=sample2(sample2_wrong, 10**5, 8, 0.75)

data = [
    Histogram(x=z[0], histnorm="probability", name="sphere1"),
    Histogram(x=z[1], histnorm="probability", name="sphere2")
]

lyt = Layout(title="Wrong direct sampling", xaxis=dict(title="pdf sphere"),
             bargroupgap=0.1)

py.plot(Figure(data=data, layout=lyt))

## 3 Naive direct ampling of n balls

The time required for sampling a correct configuration of n balls is proportional to the attempts tried, which in turn goes as $\sim 1/p$ where p is the probability for accepting the configuration. The probability to accept the i-th sphere is

$$
\begin{equation}
p_{i}=1-\dfrac{2i\sigma}{L}
\end{equation}
$$

therefore the global probability is:

$$
\begin{equation}
p=\prod_{i=1}^N p_i
\end{equation}
$$

We can also explicitly show the functional dependence on the number of trials $n$; let's put $A=L/(2\sigma)$. Then:

$$
\begin{equation}
p=\dfrac{A!A^{-n}}{(A-n)!}
\end{equation}
$$

We can point out the fact that with the parameters $L=10$ and $\sigma=0.1$ sampling the configuration space of 20 spheres start to require lot of time

In [13]:
Ns = list(range(1, 15))
times = []
for n in Ns:
    sampler = DirectPinSampler(n, 10, 0.1)
    start = time.process_time()
    sampler.sample(10)
    delta = time.process_time() - start
    times.append(delta)


py.plot([Scatter(x=Ns, y=times)])

'file://C:\\Users\\Diego\\Desktop\\computational-science\\h4\\temp-plot.html'

## 4 Deflation algorithm

The deflation algorithm is a way out of the previous impasse. We can see from the plot below its comparison with the naive strategy. The sampling time is istanteneous.

In [22]:
Ns = list(range(1, 15))
times_direct = []
times_infdef = []
for n in Ns:
    sampler_direct = DirectPinSampler(n, 20, 0.1)
    sampler_infdef = InflateDeflatePinSampler(n, 20, 0.1)

    start = time.process_time()
    sampler_direct.sample(20)
    delta = time.process_time() - start
    times_direct.append(delta)
    
    start = time.process_time()
    sampler_infdef.sample(20)
    delta = time.process_time() - start
    times_infdef.append(delta)


py.plot([
Scatter(x=Ns, y=times_direct, name="Direct"),
Scatter(x=Ns, y=times_infdef, name="Inflate-deflate"),
 ])

'file://C:\\Users\\Diego\\Desktop\\computational-science\\h4\\temp-plot.html'

## 5 Probability Distribution of n balls

In the following histogram we have portrayed the density of probabilty to have a sphere in the position X for a system of 10 hard spheres

In [23]:
sampler = InflateDeflatePinSampler(10, 20, 0.75)
s = sampler.sample(10**5).flatten()
py.plot([Histogram(x=s)])

'file://C:\\Users\\Diego\\Desktop\\computational-science\\h4\\temp-plot.html'

## 6 Analytical computation of the pdf

# ???